# Prompt Testing Notebook

Test prompts from `prompts_long_short.json` on specific policies and users.

In [1]:
import json
import pandas as pd
import os
import anthropic
from dotenv import load_dotenv

load_dotenv()

True

## Configuration
Set which policy and user to test:

In [2]:
# Configuration - modify these to test different combinations
policy_index = 0  # Which policy to test (0-based index)
user_index = 0    # Which user profile to test (0-based index)
prompt_type = "trustee"  # "delegate" or "trustee"

## Load Data

In [3]:
# Load prompts
with open('../prompts_long_short.json', 'r') as f:
    prompts = json.load(f)

# Load user profiles
written_profiles = pd.read_json("gpt-4o_written_profiles.jsonl", encoding='cp1252', lines=True)

# Load policies
policies = pd.read_json("../self_selected_policies.jsonl", lines=True)

print(f"Loaded {len(prompts)} prompt sets")
print(f"Loaded {len(written_profiles)} user profiles")
print(f"Loaded {len(policies)} policies")

Loaded 1 prompt sets
Loaded 100 user profiles
Loaded 20 policies


## Preview Selected Data

In [4]:
# Show selected policy
selected_policy = policies.iloc[policy_index].statement
print(f"Selected Policy (index {policy_index}):")
print(selected_policy)
print()

# Show selected user profile (truncated)
selected_user = written_profiles.iloc[user_index]
print(f"Selected User (ID {selected_user['ID']}):")
print(selected_user['Profile'][:200] + "...")
print()

# Show selected prompt
selected_prompt = prompts["0"][prompt_type]
print(f"Selected Prompt ({prompt_type}):")
print(selected_prompt)

Selected Policy (index 0):
The government should provide a pension for everyone over the age of 65.

Selected User (ID 1):
John Michael Rodriguez was born on March 15, 1975, in a small town in the southern United States. Growing up in a bilingual household, John was fluent in both English and Spanish, a skill that would s...

Selected Prompt (trustee):
Here is a written profile of a person that is voting on a policy proposal: {bio}
Your goal: Evaluate the **short-term** and **long-term** utility to this participant if you vote **YES** versus **NO** on this proposal.

- Score the utility out of 10, where **10 = very good for the participant** in the short/long term.
- Return a JSON object in the following structure:

{{
  "yes_vote": {{
    "reasoning": "[REASONING]",
    "short_util": [SHORT_SCORE],
    "long_util": [LONG_SCORE]
  }},
  "no_vote": {{
    "reasoning": "[REASONING]",
    "short_util": [SHORT_SCORE],
    "long_util": [LONG_SCORE]
  }}
}}


In [5]:
def run_claude(prompt, profile, policy):
    client = anthropic.Anthropic(api_key=os.environ["ANTHROPIC_API_KEY"])
    
    messages = [
        {
            "role": "user",
            "content": f"{prompt.format(bio=profile)}\n\nPolicy proposal: {policy}"
        }
    ]
    
    system = prompt.format(bio=profile)
    
    response = client.messages.create(
        model="claude-3-7-sonnet-20250219",
        max_tokens=1024,
        messages=messages,
        system=system,
        temperature=0.0
    )
    
    return response.content[0].text

## Run Test

In [6]:
# Run the test
response = run_claude(selected_prompt, selected_user['Profile'], selected_policy)

print("Claude Response:")
print("=" * 50)
print(response)
print("=" * 50)

Claude Response:
```json
{
  "yes_vote": {
    "reasoning": "John is currently 48 years old, so he's approaching retirement age in the next 15-20 years. As someone with a modest income ($30,000-$49,999) who had to leave college early due to financial constraints, he likely hasn't been able to save extensively for retirement. A government pension would provide him with additional financial security in his later years. While his Republican political identity might make him skeptical of government programs, the direct financial benefit to him and his family would likely outweigh ideological concerns. In the short term, there's little immediate benefit as he's not yet eligible, but knowing this safety net will exist could reduce financial stress.",
    "short_util": 3,
    "long_util": 8
  },
  "no_vote": {
    "reasoning": "If John votes against the pension proposal, this aligns with his Republican identity and belief in limited government. In the short term, this ideological consistency 

## Parse JSON Response (if applicable)

In [7]:
# Try to parse JSON response
try:
    # Extract JSON from response
    json_start = response.find('{')
    json_end = response.rfind('}') + 1
    json_str = response[json_start:json_end]
    
    parsed_response = json.loads(json_str)
    
    print("Parsed JSON Response:")
    print(json.dumps(parsed_response, indent=2))
    
except (json.JSONDecodeError, ValueError) as e:
    print(f"Could not parse JSON: {e}")
    print("Raw response shown above.")

Parsed JSON Response:
{
  "yes_vote": {
    "reasoning": "John is currently 48 years old, so he's approaching retirement age in the next 15-20 years. As someone with a modest income ($30,000-$49,999) who had to leave college early due to financial constraints, he likely hasn't been able to save extensively for retirement. A government pension would provide him with additional financial security in his later years. While his Republican political identity might make him skeptical of government programs, the direct financial benefit to him and his family would likely outweigh ideological concerns. In the short term, there's little immediate benefit as he's not yet eligible, but knowing this safety net will exist could reduce financial stress.",
    "short_util": 3,
    "long_util": 8
  },
  "no_vote": {
    "reasoning": "If John votes against the pension proposal, this aligns with his Republican identity and belief in limited government. In the short term, this ideological consistency mig